In [37]:
import torch
import supervision as sv
import cv2
from ultralytics import YOLO
print(torch.__version__)        # should show +cu128
print(torch.version.cuda)       # should match runtime CUDA version
print(torch.cuda.is_available())  # True if GPU detected

2.9.1+cu128
12.8
True


In [39]:
VIDEO_PATH = "../VIDEOSYOLO/video3.mp4"

model = YOLO("yolo11n.pt")

frames_generator = sv.get_video_frames_generator(
    source_path=VIDEO_PATH,
)

box_annotator = sv.BoxAnnotator()

for idx, frame in enumerate(frames_generator):
    result = model(frame, device="cuda")[0]
    detections = sv.Detections.from_ultralytics(result)
    print(detections)

    # Dibujar cajas
    annotated_frame = frame.copy()
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame,
        detections=detections
    )

    # Mostrar frame anotado
    cv2.imshow("Annotated Frame", annotated_frame)

    # salir con 'q'
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()



0: 384x640 8 cars, 5.2ms
Speed: 0.8ms preprocess, 5.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Detections(xyxy=array([[      200.7,      72.328,      225.55,      102.57],
       [      177.2,      38.181,      196.61,      56.198],
       [     142.82,      40.111,      162.01,      60.049],
       [     284.12,      97.587,      312.22,      134.74],
       [     297.21,      144.51,      337.36,      201.51],
       [     111.92,      136.68,      159.96,      182.07],
       [     57.003,      336.62,      123.91,      359.63],
       [     198.97,      1.1514,      211.26,      11.298]], dtype=float32), mask=None, confidence=array([    0.74128,     0.69268,     0.64146,     0.63126,     0.59453,     0.58544,     0.56167,     0.35443], dtype=float32), class_id=array([2, 2, 2, 2, 2, 2, 2, 2]), tracker_id=None, data={'class_name': array(['car', 'car', 'car', 'car', 'car', 'car', 'car', 'car'], dtype='<U3')}, metadata={})

0: 384x640 7 cars, 5.6ms
Speed: 0.9